<a href="https://colab.research.google.com/github/A1maan/smail-vlm/blob/almaan/simple-inference-vlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>